## Imports

In [11]:
# General imports
import time
import json
import random

# Intent recognition imports
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# ChatGPT imports
%load_ext autoreload
%autoreload 2
from gpt import GPT

# Chat demo
import gradio as gr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Intent recognition model

In [14]:
# Load the data from JSON file
with open("intent_recognition.json", "r") as f:
    data = json.load(f)

# Define the training data
train_data = []
for datum in data:
    texts = datum["text"]
    label = datum["intent"]
    for text in texts:
        train_data.append((text, label))

# Define the mapping between top-level labels and integers
label_map = {label: i for i, label in enumerate(set([data[1] for data in train_data]))}

# Convert the training data labels to integers using the label_map
labels = torch.tensor([label_map[data[1]] for data in train_data])

# Load the pre-trained BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_map))
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the training data and convert to tensors
inputs = tokenizer.batch_encode_plus([data[0] for data in train_data], padding=True, truncation=True, return_tensors="pt")

# Fine-tune the model on the training data
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4) #default 5e-5
loss_fn = torch.nn.CrossEntropyLoss()
for epoch in range(20):
    optimizer.zero_grad()
    outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    # Evaluate the model on the training data
    predictions = outputs.logits.argmax(axis=1)
    accuracy = (predictions == labels).sum()



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1, Loss: 3.8007309436798096
Epoch 2, Loss: 3.7969255447387695
Epoch 3, Loss: 3.652984857559204
Epoch 4, Loss: 3.7080342769622803
Epoch 5, Loss: 3.635124921798706
Epoch 6, Loss: 3.378645420074463
Epoch 7, Loss: 3.0341482162475586
Epoch 8, Loss: 3.211941957473755
Epoch 9, Loss: 2.8290586471557617
Epoch 10, Loss: 2.6684978008270264
Epoch 11, Loss: 2.477515697479248
Epoch 12, Loss: 2.1890857219696045
Epoch 13, Loss: 1.9279072284698486
Epoch 14, Loss: 1.7251923084259033
Epoch 15, Loss: 1.5288735628128052
Epoch 16, Loss: 1.344827651977539
Epoch 17, Loss: 1.1490167379379272
Epoch 18, Loss: 0.9751096963882446
Epoch 19, Loss: 0.8410384654998779
Epoch 20, Loss: 0.7184804081916809


## Save and load saved model

In [15]:
# Save model after training it
# torch.save(model, "intent_recognition.pt")

# Load the model from a file
# model = torch.load("intent_recognition.pt")

## Query intent function

In [16]:
def get_intent(question):

    # Tokenize the test question and convert to tensors
    inputs = tokenizer.encode_plus(question, padding=True, truncation=True, return_tensors="pt")

    # Get the model's prediction for the test question
    with torch.no_grad():
        outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"])
    predictions = outputs.logits.argmax(axis=1)
    predicted_label = list(label_map.keys())[list(label_map.values()).index(predictions[0].item())]
    print(f"Predicted intent: {predicted_label}")
    return predicted_label

# Sample test question
test_question = "Can you summarize the legal statement?"
get_intent(test_question)


Predicted intent: liability_legal_statement


'liability_legal_statement'

## ChatGPT model

In [25]:
# GPT model here
# gpt_model = GPT()

gpt_model = False

# Test ChatGPT model
# gpt_model.answer_question(question='What is the most important thing I need to know about your privacy statement?')


## Get response using intent

In [22]:
def get_object_by_intent(intent):
    for object in data:
        if object['intent'] == intent:
            return object
    return None

## Loading user data

In [27]:
# Load user_profiles.json
with open('user_profiles.json', 'r') as file:
    user_profiles = json.load(file)

# Print the loaded data
print(user_profiles)


[{'user_id': '0', 'profile_published': False, 'open_to_connecting': False, 'visibile_in_search': False, 'findable_to': 'logged_on_only', 'individual_user_viewability': [{'user_id': '1', 'profile_card_component_0_visible': True, 'profile_card_component_1_visible': True, 'profile_card_component_2_visible': True, 'profile_card_component_3_visible': True, 'profile_card_component_4_visible': True}, {'user_id': '2', 'profile_card_component_0_visible': True, 'profile_card_component_1_visible': False, 'profile_card_component_2_visible': False, 'profile_card_component_3_visible': False, 'profile_card_component_4_visible': False}]}, {'user_id': '1', 'profile_published': True, 'open_to_connecting': True, 'visibile_in_search': True, 'findable_to': 'everyone'}, {'user_id': '2', 'profile_published': False, 'open_to_connecting': False, 'visibile_in_search': False, 'findable_to': 'logged_on_only'}, {'user_id': '3', 'profile_published': False, 'open_to_connecting': False, 'visibile_in_search': False, '

## Loading user security levels

In [31]:
with open('user_security_levels.json', 'r') as file:
    user_security_levels = json.load(file)

# Accessing the values
low_security = user_security_levels['low_security']
medium_security = user_security_levels['medium_security']
high_security = user_security_levels['high_security']

# Example usage
print(low_security)
print(medium_security)
print(high_security)

print(print(low_security['profile_published'])))

{'profile_published': True, 'open_to_connecting': True, 'visibile_in_search': True, 'findable_to': 'everyone'}
{'profile_published': False, 'open_to_connecting': True, 'visibile_in_search': True, 'findable_to': 'logged_on_only'}
{'profile_published': False, 'open_to_connecting': False, 'visibile_in_search': False, 'findable_to': 'logged_on_only'}
True
None


## Functions to edit user data

In [ ]:
def estimate_user_security_level(user_id):
    '''
    Set up this function to ask the user a bunch of questions, and then based on the answers choose a secutiry level that suits them
    then use the above pre-defined user profile security levels to set their security settings
    since this will be a rule based approach it should just be a bunch of if statements and use the gradio chat to communicate with the user
    '''
    return None

def update_user_setting(user_id, update_data):
    '''
    This function should be called with the user_id and the setting that should be updated, the answer can be processed here to determine the new value it should have
    or it can be done else where as long as the actual update is done here. This can be done by using a json functionality to update the file
    '''
    return None

def set_user_calling_card_visibility(user_id, second_user_id):
    '''
    This function should be called with the user_id and the second_user_id and the calling card visibility should be updated
    this can be done by using a json functionality to update the file. As can been seen in the user profiles json, each user has an array that contains objects
    each of the objects contains the id of the user who's the settings are for, as well as the personal visbility for the target user

    Example:
    User Frank (id: 0) want User (Lukas id: 1) to only be able to see select elements of their calling card. This function does that by taking the user id's and updating the
    calling card visiblities accordingly to the intent that's recognised
    '''
    return None

# Set up gradio chat for demo

In [26]:
theme = gr.themes.Soft(
    primary_hue="orange",
    secondary_hue="orange",
).set(
    body_text_color_dark='*neutral_800',
    background_fill_primary_dark='*neutral_50',
    background_fill_secondary_dark='*neutral_50',
    border_color_accent_dark='*primary_300',
    border_color_primary_dark='*neutral_200',
    color_accent_soft_dark='*primary_50',
    link_text_color_dark='*secondary_600',
    link_text_color_active_dark='*secondary_600',
    link_text_color_hover_dark='*secondary_700',
    link_text_color_visited_dark='*secondary_500',
    block_background_fill='*neutral_100',
    block_background_fill_dark='*neutral_100',
    block_label_background_fill='*primary_400',
    block_label_background_fill_dark='*primary_400',
    block_label_text_color='*neutral_50',
    block_label_text_color_dark='*neutral_50',
    block_title_text_color='*neutral_50',
    block_title_text_color_dark='*neutral_50',
    checkbox_background_color_dark='*background_fill_primary',
    checkbox_background_color_selected='*primary_500',
    checkbox_background_color_selected_dark='*primary_500',
    checkbox_border_color_dark='*neutral_100',
    checkbox_border_color_focus='*primary_300',
    checkbox_border_color_focus_dark='*primary_300',
    checkbox_border_color_hover_dark='*neutral_300',
    checkbox_border_color_selected='*primary_500',
    checkbox_border_color_selected_dark='*primary_500',
    checkbox_border_width_dark='1px',
    checkbox_label_background_fill_selected_dark='*primary_500',
    checkbox_label_text_color_selected_dark='white',
    error_background_fill_dark='#fee2e2',
    error_border_color_dark='#fecaca',
    input_background_fill_dark='white',
    input_background_fill_focus_dark='*secondary_500',
    input_border_color_dark='*neutral_50',
    input_border_color_focus_dark='*secondary_300',
    input_placeholder_color_dark='*neutral_400',
    slider_color_dark='*primary_500',
    stat_background_fill_dark='*primary_300',
    table_border_color_dark='*neutral_300',
    table_even_background_fill_dark='white',
    table_odd_background_fill_dark='*neutral_50',
    button_primary_background_fill_dark='*primary_500',
    button_primary_background_fill_hover_dark='*primary_400',
    button_primary_border_color_dark='*primary_200',
    button_secondary_background_fill_dark='white',
    button_secondary_background_fill_hover_dark='*neutral_100',
    button_secondary_border_color_dark='*neutral_200',
    button_secondary_text_color_dark='*neutral_800'
)

with gr.Blocks(theme=theme, css="chat/chat.css") as demo:
    gr.Image("https://iyyu.com/_nuxt/img/navbar_logoW@2x.79eba99.png", interactive=False, tool="image", show_label=False, elem_classes="logo").style(width=200)
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    submit = gr.Button("Submit")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]

        intent = get_intent(question=user_message)
        print('intent:', intent)
        
        intent =  get_object_by_intent(intent)
        
        # generating a response with GPT if the main intent was 'privacy_policy' or 'legal_statement'
        use_gpt = intent['use_gpt']

        # if the intent is 'privacy_policy' or 'legal_statement', use GPT to generate a response
        response = '(ChatGPT - intent found: {}): \n{}'.format(intent['intent'], gpt_model.answer_question(question=user_message)) if use_gpt else '(intent found: {}): \n{}'.format(intent['intent'], intent['responses'][0])

        # response = random.choice(response_map[intent])
        history[-1][1] = response
        # The sleep is to simulate a more natural conversation
        time.sleep(1)
        return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    submit.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    
demo.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Predicted intent: opt_out_targeted_advertising
intent: opt_out_targeted_advertising


Traceback (most recent call last):
  File "c:\Program Files\Python38\lib\site-packages\gradio\routes.py", line 395, in run_predict
    output = await app.get_blocks().process_api(
  File "c:\Program Files\Python38\lib\site-packages\gradio\blocks.py", line 1193, in process_api
    result = await self.call_function(
  File "c:\Program Files\Python38\lib\site-packages\gradio\blocks.py", line 916, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Program Files\Python38\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Program Files\Python38\lib\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "c:\Program Files\Python38\lib\site-packages\anyio\_backends\_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "C:\Users\Frank\AppData\Local\Temp\ipykernel_16896\2964325488.py", line 77, in bot
    response